In [7]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import gls
import math
import time

In [8]:
X_train = pd.read_csv('../data/curated/model/X_train.csv')
y_train = pd.read_csv('../data/curated/model/y_train.csv')
X_val = pd.read_csv('../data/curated/model/X_val.csv')
y_val = pd.read_csv('../data/curated/model/y_val.csv')
X_test = pd.read_csv('../data/curated/model/X_test.csv')
y_test = pd.read_csv('../data/curated/model/y_test.csv')

In [9]:
X_train

,Unnamed: 0,PULocationID,total_with_tip,trip_distance,fare_amount,tip_amount,TEMP,DEWP,WDSP,MXSPD,GUST,MAX,MIN
0,0,1,1.084904,3.372621,1.363832,0.931607,0.798488,0.676687,1.331435,2.467679,-0.442229,0.881842,0.814166
1,1,1,3.201487,-2.551986,2.712761,1.665827,1.331756,0.560247,0.685311,-0.368222,-0.479152,1.316808,1.355327
2,2,1,1.123441,2.897021,1.000658,0.974456,1.235887,0.653399,0.513011,1.580107,-0.455680,1.208066,1.138863
3,3,1,-1.449377,0.260843,-1.178380,0.206265,1.122043,1.289936,-0.391562,0.281221,-0.468867,1.208066,1.138863
4,4,1,-0.193871,1.877880,0.455899,0.417579,1.259854,-0.689540,-0.434637,-0.563054,-0.495241,1.534291,0.814166
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18364,18364,163,0.721619,-0.528510,-0.446066,0.974456,-0.705447,-0.006426,-1.209986,-1.039312,2.110242,-0.749279,-0.484620
18365,18365,163,0.643079,-0.554441,-0.445522,0.943030,-0.507719,0.436045,-0.736162,-0.822832,-0.495241,-0.423055,-0.268156
18366,18366,163,0.196409,-0.468254,-0.591596,0.713271,-0.124245,0.583535,-0.994611,-0.151741,-0.471504,-0.314314,0.164773
18367,18367,163,0.092336,-0.367722,-0.680809,0.709430,-0.507719,0.257504,0.383786,1.147145,-0.455680,-0.640538,-0.376388


In [10]:
# Predictors
X_train = X_train.iloc[:, 1:]
X_val = X_val.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

# Responses
y_train = y_train.iloc[:, 1:]
y_val = y_val.iloc[:, 1:]
y_test = y_test.iloc[:, 1:]


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# hyperparameter grid
rf_param_grid = {
    "n_estimators": [50, 60],
    "max_depth": list(range(10, 80, 10)),
    # adjust as needed for fine-tuning
    # "min_samples_leaf": list(range(1, 10)),
    # "min_samples_split": list(range(2, 10)),
    "max_features": [0.5, 0.75],
    "max_samples": [0.5, 0.75],
    "ccp_alpha": [0, 0.01, 0.001, 1, 10]
}

rf_regressor = RandomForestRegressor()

grid_search = GridSearchCV(
    estimator=rf_regressor, 
    param_grid=rf_param_grid, 
    cv=5, 
    n_jobs=-1,
    scoring='r2'  
)

# to fit model and find the best parameters
# grid_search.fit(X_train, y_train)

In [6]:


start_time = time.time()
grid_search.fit(X_train, y_train)
execution_time = round(time.time() - start_time, 3)

print(f"Grid search completed in: {execution_time:.3f} seconds")
print(f"Best parameters found: {grid_search.best_params_}")


/home/ambrosehe/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ambrosehe/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ambrosehe/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ambrosehe/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

Grid search completed in: 1060.285 seconds
Best parameters found: {'ccp_alpha': 1, 'max_depth': 50, 'max_features': 0.75, 'max_samples': 0.5, 'n_estimators': 50}


In [12]:
from sklearn.ensemble import RandomForestRegressor

start_time = time.time()

rf_clf = RandomForestRegressor(
    n_estimators=50, 
    max_depth=50, 
    max_features=0.75,
    ccp_alpha=1, 
    max_samples=0.5
)
rf_clf.fit(X_train, y_train)

execution_time = round(time.time() - start_time, 3)
print(f"Execution time of RandomForestRegressor: {execution_time} s")


/home/ambrosehe/.local/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Execution time of RandomForestRegressor: 13.263 s


In [13]:
from sklearn import metrics
import numpy as np

rf_train_predictions = rf_clf.predict(X_train)

mae_train = metrics.mean_absolute_error(y_train, rf_train_predictions)
mse_train = metrics.mean_squared_error(y_train, rf_train_predictions)
rmse_train = np.sqrt(mse_train)

print("Performance Metrics for Training Set")
print("-" * 40)
print(f"{'Metric':<30} {'Value':>10}")
print(f"{'-' * 40}")
print(f"{'Mean Absolute Error':<30} {mae_train:>10.3f}")
print(f"{'Mean Squared Error':<30} {mse_train:>10.3f}")
print(f"{'Root Mean Squared Error':<30} {rmse_train:>10.3f}")


Performance Metrics for Training Set
----------------------------------------
Metric                              Value
----------------------------------------
Mean Absolute Error                 0.970
Mean Squared Error                 23.341
Root Mean Squared Error             4.831


In [14]:
X_train, X_val = X_train.align(X_val, join='left', axis=1)

# predicting values on the validation set
rf_val_predictions = rf_clf.predict(X_val)

mae_val = metrics.mean_absolute_error(y_val, rf_val_predictions)
mse_val = metrics.mean_squared_error(y_val, rf_val_predictions)
rmse_val = np.sqrt(mse_val)

print("Performance Metrics for Validation Set")
print("-" * 40)
print(f"{'Metric':<30} {'Value':>10}")
print(f"{'-' * 40}")
print(f"{'Mean Absolute Error':<30} {mae_val:>10.3f}")
print(f"{'Mean Squared Error':<30} {mse_val:>10.3f}")
print(f"{'Root Mean Squared Error':<30} {rmse_val:>10.3f}")


Performance Metrics for Validation Set
----------------------------------------
Metric                              Value
----------------------------------------
Mean Absolute Error                 4.093
Mean Squared Error                 97.574
Root Mean Squared Error             9.878


In [15]:
rf_test_predict = rf_clf.predict(X_test)

mae_test = metrics.mean_absolute_error(y_test, rf_test_predict)
mse_test = metrics.mean_squared_error(y_test, rf_test_predict)
rmse_test = np.sqrt(mse_test)

print("Performance Metrics for Test Set")
print("-" * 40)
print(f"{'Metric':<30} {'Value':>10}")
print(f"{'-' * 40}")
print(f"{'Mean Absolute Error':<30} {mae_test:>10.3f}")
print(f"{'Mean Squared Error':<30} {mse_test:>10.3f}")
print(f"{'Root Mean Squared Error':<30} {rmse_test:>10.3f}")

Performance Metrics for Test Set
----------------------------------------
Metric                              Value
----------------------------------------
Mean Absolute Error                 4.023
Mean Squared Error                 49.268
Root Mean Squared Error             7.019


In [16]:
predictions_df = pd.DataFrame({
    'Index': X_test.index,  # include index here if needed
    'Predicted': rf_test_predict
})

predictions_df.to_csv('../data/curated/model/rf_predictions.csv', index=False)